<a href="https://colab.research.google.com/github/Heity94/TWSM_Lab/blob/main/Project/Notebooks/MA_FineTuneSentenceTransformer_SynonymDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U sentence-transformers -q
from sentence_transformers import SentenceTransformer

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation
from torch.utils.data import DataLoader
import pandas as pd
import math
import logging
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_path = "/content/drive/MyDrive/2022_Analytics Lab Student Projects/Data/Topic 1/Data_Team1/SynDet/Synonym_Dataset/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Loading the train dataset
train = pd.read_csv(data_path+"SynDet_ontology_train_syn.csv", index_col=0)

In [ ]:
train.head()

,entity_id,category,label,synonym,is_synonym
235036,individual participant,study object,PARTICIPANTS,usable and information agent,1
189726,individual participant,study object,PARTICIPANTS,professional and senior research executives,1
291037,individual participant,study object,PARTICIPANTS,business and IS IS informant,1
31468,survey,research method,COLLECTION_METHOD,telephone based postal questionnaire descriptions,1
288690,individual participant,study object,PARTICIPANTS,business and professional individual,1


In [ ]:
eval = pd.read_csv(data_path+"SynDet_ontology_val_syn.csv", index_col=0)
eval.head()

,entity_id,category,label,synonym,is_synonym
112895,graph database,domain specific entity,TECHNOLOGY,information security practice,0
73975,conjoint analysis,methodological entity,ANALYSIS_METHOD,statistical metric,0
87794,experimental design,research method,COLLECTION_METHOD,simulation,0
15021,communication service infrastructure,domain specific entity,TECHNOLOGY,internet aided communication electronic data p...,1
8554,IS downsizing,domain specific entity,TOPIC,downsize of internets,1


In [ ]:
test = pd.read_csv(data_path+"SynDet_ontology_test_syn.csv", index_col=0)
test.head()

,entity_id,category,label,synonym,is_synonym
58723,banking sytem,domain specific entity,TECHNOLOGY,mySAP,0
19994,government,domain specific entity,SECTOR,data breach,0
64267,Orkut,domain specific entity,TECHNOLOGY,RSA signature,0
68597,macOS,domain specific entity,TECHNOLOGY,telecommunications equipment industry,0
13590,individual participant,study object,PARTICIPANTS,business and executive security response,1


In [ ]:
test['is_synonym'].dtype

dtype('int64')

In [ ]:
train_samples = []
for index, row in train.iterrows():
  inp_example = InputExample(texts=[row['entity_id'], row['synonym']], label=float(row['is_synonym']))
  train_samples.append(inp_example)

In [ ]:
evaluation_samples = []
for index, row in eval.iterrows():
  inp_example = InputExample(texts=[row['entity_id'], row['synonym']], label=float(row['is_synonym']))
  evaluation_samples.append(inp_example)

In [ ]:
test_samples = []
for index, row in test.iterrows():
  inp_example = InputExample(texts=[row['entity_id'], row['synonym']], label=float(row['is_synonym']))
  test_samples.append(inp_example)

In [ ]:
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model=model)

In [ ]:
num_epochs = 2
warmup_steps = float(math.ceil(len(train_dataloader) * num_epochs * 0.1)) #10% of train data for warm-up
#logging.info("Warmup-steps: {}".format(warmup_steps))

In [ ]:
warmup_steps

7093.0

In [ ]:
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(evaluation_samples)

In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)], 
          epochs=num_epochs, 
          warmup_steps=warmup_steps, 
          evaluator=evaluator, 
          evaluation_steps=5000, 
          output_path=data_path)

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/35465 [00:00<?, ?it/s]

In [ ]:
model = SentenceTransformer(data_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples)
test_evaluator(model, output_path=data_path)